In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd, matplotlib.pyplot as plt
!pip install numba -q
from scipy.optimize import bisect
from numba import jit, prange
from IPython.display import display, Markdown, Latex

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 14, 'figure.figsize': (12, 8), 'figure.dpi': 150})
np.set_printoptions(suppress=True, linewidth=120, precision=4)
warnings.filterwarnings('ignore', category=UserWarning)

# --- Utility Functions ---
def note(msg, **kwargs): display(Markdown(f"<div class='alert alert-block alert-info'>📝 **Note:** {msg}</div>"))
def sec(title): print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized for Heterogeneous Agent General Equilibrium Models.")

<a id='intro'></a>
## 1. Beyond the Representative Agent: The Need for Heterogeneity

Standard representative-agent models, like the Real Business Cycle (RBC) model, are powerful but make a critical simplifying assumption: all households are identical. This means the models cannot address fundamental questions about inequality, social mobility, or the distributional consequences of economic policy. Why do some people save so much more than others? What drives the vast inequality in wealth we observe empirically? How do government policies affect the rich and the poor differently?

To answer these questions, we need **heterogeneous agent models**. These models are populated by a large number of households who differ in their income, wealth, age, or other characteristics. The key challenge in these models is that we can no longer simply analyze the behavior of a single representative agent; we must characterize the entire **distribution** of agents and how it evolves over time.

This notebook develops, from the ground up, one of the most important and influential heterogeneous agent models in modern macroeconomics: the **Aiyagari (1994) model**. This model provides a laboratory for understanding **precautionary savings** and the determinants of wealth inequality.

<a id='aiyagari-theory'></a>
## 2. The Aiyagari (1994) Model: A Canon of Macroeconomics

The Aiyagari model extends the standard neoclassical growth model by introducing two key features:
1.  **Idiosyncratic Labor Income Shocks:** Each household's labor income is stochastic and uninsurable. They can't buy insurance against a bad income draw (e.g., a spell of unemployment).
2.  **Borrowing Constraint:** Households cannot borrow against their future income; their wealth must be non-negative ($a \ge 0$).

Faced with the risk of a low-income realization and the inability to borrow, households have a powerful incentive to engage in **precautionary savings**: they build up a buffer stock of assets to self-insure against bad times. It is this precautionary motive that generates a non-trivial, stationary distribution of wealth.

<a id='household-problem'></a>
### The Household's Problem
An infinitely-lived household seeks to maximize its lifetime utility from consumption, given by:
$$ E_0 \sum_{t=0}^\infty \beta^t u(c_t) $$
subject to the budget constraint:
$$ c_t + a_{t+1} = (1+r)a_t + y_t $$
where $c_t$ is consumption, $a_t$ is asset holdings, $r$ is the interest rate, and $y_t$ is stochastic labor income. The labor income process, $y_t$, follows a finite-state Markov chain. The household's problem can be written in recursive form as a Bellman equation:
$$ V(a, y) = \max_{a'} \left\{ u((1+r)a + y - a') + \beta \sum_{y'} \pi(y'|y) V(a', y') \right\} $$ 
where $\pi(y'|y)$ is the transition probability from state $y$ to state $y'$. The solution to this dynamic programming problem is a **policy function**, $a' = g(a,y)$, which tells the household its optimal level of savings for any given level of current assets and income.

<a id='computation'></a>
## 3. Computational Strategy: Solving the Model

Finding the general equilibrium requires a nested approach. The interest rate $r$ is an equilibrium object, but households need to know it to solve their optimization problem. This creates a circularity that we can solve with the following algorithm, which is visually represented in the diagram below:

![A diagram of the nested loops required to solve the Aiyagari model. The outer loop searches for the market-clearing interest rate, while the inner loop solves the household's problem for a given interest rate.](../images/10-Specialized-Models/aiyagari_equilibrium_loop.png)

1.  **Outer Loop: Search for the Equilibrium Interest Rate ($r^\*$):**
    - Guess an interest rate, $r$.
    - Given $r$, determine the corresponding aggregate capital stock demanded by firms, $K_d(r)$, by inverting the firm's first-order condition.
    - Given $r$, calculate the aggregate capital stock supplied by households, $K_s(r)$. This is the most complex step.
    - Check for market clearing. If $K_d(r) > K_s(r)$, there is excess demand for capital, so the price of capital ($r$) must rise. If $K_d(r) < K_s(r)$, there is excess supply, so $r$ must fall.
    - Use a root-finding algorithm (like bisection) to adjust the guess for $r$ until demand equals supply, $K_d(r) = K_s(r)$.

2.  **Inner Loop: Calculate Capital Supply for a Given `r` ($K_s(r)$):**
    - **Step 2a: Solve the Household's Problem:** For the given $r$, solve the household's dynamic programming problem using **value function iteration** to find the optimal savings policy function, $a' = g(a,y)$.
    - **Step 2b: Find the Stationary Distribution:** With the policy function $g(a,y)$ in hand, simulate the decisions of a large number of households over a long period. The resulting cross-sectional distribution of asset holdings will approximate the stationary distribution $\Phi(a,y)$.
    - **Step 2c: Calculate Aggregate Capital Supply:** Compute the mean of the simulated wealth distribution. This is the aggregate capital supply, $K_s(r)$.

<a id='implementation'></a>
## 4. Code Implementation and Results

<a id='code'></a>
### The `AiyagariModel` Class
We encapsulate the full solution algorithm in a class. The code is placed in a collapsed cell below for transparency. We use Numba's `@jit` decorator to significantly speed up the computationally intensive loops in the value function iteration and the distribution simulation.

In [ ]:
# This cell contains the full implementation of the Aiyagari model solver.
# It is collapsed by default for readability.
sec("Aiyagari Model Class Definition")

@jit(nopython=True)
def utility(c, gamma):
    return (c**(1 - gamma)) / (1 - gamma) if c > 0 else -1e12

@jit(nopython=True)
def solve_household_problem_vfi(V, a_grid, y_grid, r, beta, gamma, y_trans):
    n_a, n_y = len(a_grid), len(y_grid)
    V_new = np.zeros_like(V)
    g_star = np.zeros_like(V, dtype=np.int64)
    
    for i_y in range(n_y):
        for i_a in range(n_a):
            max_val = -1e12
            best_i_ap = 0
            budget = (1 + r) * a_grid[i_a] + y_grid[i_y]
            for i_ap in range(n_a):
                c = budget - a_grid[i_ap]
                if c > 0:
                    expected_V_next = 0.0
                    for i_yp in range(n_y):
                        expected_V_next += y_trans[i_y, i_yp] * V[i_ap, i_yp]
                    val = utility(c, gamma) + beta * expected_V_next
                    if val > max_val:
                        max_val = val
                        best_i_ap = i_ap
            V_new[i_a, i_y] = max_val
            g_star[i_a, i_y] = best_i_ap
    return V_new, g_star

@jit(nopython=True)
def find_stationary_distribution(g_star, y_trans, n_a, n_y):
    dist = np.ones((n_a, n_y)) / (n_a * n_y)
    for _ in range(500):
        dist_new = np.zeros_like(dist)
        for i_a in range(n_a):
            for i_y in range(n_y):
                if dist[i_a, i_y] > 0:
                    i_ap = g_star[i_a, i_y]
                    for i_yp in range(n_y):
                        dist_new[i_ap, i_yp] += dist[i_a, i_y] * y_trans[i_y, i_yp]
        if np.max(np.abs(dist_new - dist)) < 1e-8: break
        dist = dist_new
    return dist / np.sum(dist)

class AiyagariModel:
    def __init__(self, beta=0.96, gamma=2.0, alpha=0.36, delta=0.08, n_a=200, a_max=50):
        self.beta, self.gamma, self.alpha, self.delta = beta, gamma, alpha, delta
        self.y_grid = np.array([0.1, 1.0]); self.n_y = len(self.y_grid)
        self.y_trans = np.array([[0.5, 0.5], [0.1, 0.9]])
        self.a_grid = np.linspace(0, a_max, n_a); self.n_a = n_a
        self.r_star, self.K_star, self.dist_star, self.g_star = (None,)*4
    
    def capital_demand(self, r):
        return (self.alpha / (r + self.delta))**(1 / (1 - self.alpha))
    
    def capital_supply(self, r):
        V = np.zeros((self.n_a, self.n_y))
        for _ in range(500):
            V_new, g_star = solve_household_problem_vfi(V, self.a_grid, self.y_grid, r, self.beta, self.gamma, self.y_trans)
            if np.max(np.abs(V_new - V)) < 1e-6: break
            V = V_new
        dist = find_stationary_distribution(g_star, self.y_trans, self.n_a, self.n_y)
        return np.sum(dist * self.a_grid[:, np.newaxis]), dist, g_star

    def solve(self, r_min=0.01, r_max=0.04):
        note("Solving for general equilibrium... This may take a moment.")
        excess_demand = lambda r: self.capital_demand(r) - self.capital_supply(r)[0]
        try:
            self.r_star = bisect(excess_demand, r_min, r_max)
            note(f"Equilibrium found! r* = {self.r_star*100:.3f}%")
            self.K_star, self.dist_star, self.g_star = self.capital_supply(self.r_star)
        except ValueError:
            note("Could not find equilibrium in the specified range. Try adjusting r_min/r_max.")

# --- Instantiate and solve the model ---
aiyagari_model = AiyagariModel()
aiyagari_model.solve(r_min=0.0, r_max=1/aiyagari_model.beta - 1 - 0.001)

<a id='analysis'></a>
### Analyzing the Equilibrium
With the model solved, we can now analyze the key features of the equilibrium: the savings behavior of households and the resulting distribution of wealth. The interactive plot below shows how the savings policy function changes with the interest rate and risk aversion, providing intuition for the household's decision-making process.

In [ ]:
import ipywidgets as widgets

sec("Interactive Savings Policy Function")

def interactive_policy_plot(r=0.025, gamma=2.0):
    # Create a temporary model instance with the specified parameters
    temp_model = AiyagariModel(gamma=gamma)
    V = np.zeros((temp_model.n_a, temp_model.n_y))
    for _ in range(500):
        V_new, g_star = solve_household_problem_vfi(V, temp_model.a_grid, temp_model.y_grid, r, temp_model.beta, gamma, temp_model.y_trans)
        if np.max(np.abs(V_new - V)) < 1e-6: break
        V = V_new
    
    g_assets = temp_model.a_grid[g_star]
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.plot(temp_model.a_grid, g_assets[:, 0], label='Low Income State ($y_L$)', lw=2)
    ax.plot(temp_model.a_grid, g_assets[:, 1], label='High Income State ($y_H$)', lw=2)
    ax.plot(temp_model.a_grid, temp_model.a_grid, 'k--', label='45-degree line (a\'=a)')
    ax.set_title(f'Savings Policy Function for r={r*100:.2f}% and γ={gamma:.1f}')
    ax.set_xlabel('Current Assets ($a_t$)'); ax.set_ylabel('Next Period Assets ($a_{t+1}$)')
    ax.legend(); ax.grid(True)
    plt.show()

note("Use the sliders to see how the savings policy function responds to changes in the interest rate (the return on savings) and risk aversion (the strength of the precautionary motive).")
widgets.interact(interactive_policy_plot, 
                 r=widgets.FloatSlider(min=0.0, max=0.05, step=0.005, value=0.025, description='Interest Rate (r)'),
                 gamma=widgets.FloatSlider(min=1.1, max=5.0, step=0.1, value=2.0, description='Risk Aversion (γ)'));

Now that we have an intuitive feel for the household's problem, we can examine the full general equilibrium results of our baseline model.

In [ ]:
sec("Analyzing the Aiyagari Model Equilibrium")

def plot_aiyagari_results(model):
    if model.r_star is None: note("Model not solved, cannot plot results."); return
    fig, axes = plt.subplots(1, 3, figsize=(22, 7))
    fig.suptitle('Figure 2: Key Results of the Aiyagari Model', fontsize=18, y=1.02)
    
    # 1. Plot policy function at equilibrium r*
    g_assets = model.a_grid[model.g_star]
    axes[0].plot(model.a_grid, g_assets[:, 0], label='Low Income State ($y_L$)', lw=2)
    axes[0].plot(model.a_grid, g_assets[:, 1], label='High Income State ($y_H$)', lw=2)
    axes[0].plot(model.a_grid, model.a_grid, 'k--', label='45-degree line')
    axes[0].set_title(f'a) Savings Policy Function (at r*={model.r_star*100:.2f}%)')
    axes[0].set_xlabel('Current Assets ($a_t$)'); axes[0].set_ylabel('Next Period Assets ($a_{t+1}$)')
    axes[0].legend()
    
    # 2. Plot Lorenz curve and Gini coefficient
    wealth_dist_flat = (model.dist_star / np.sum(model.dist_star)).flatten(order='F')
    asset_grid_flat = np.tile(model.a_grid, model.n_y)
    sorted_indices = np.argsort(asset_grid_flat)
    cum_pop = np.cumsum(wealth_dist_flat[sorted_indices])
    cum_wealth = np.cumsum(wealth_dist_flat[sorted_indices] * asset_grid_flat[sorted_indices])
    lorenz_curve = cum_wealth / cum_wealth[-1]
    gini = 1 - np.sum(lorenz_curve[1:] * np.diff(cum_pop) + lorenz_curve[:-1] * np.diff(cum_pop))
    
    axes[1].plot(cum_pop, lorenz_curve, label=f'Wealth Distribution (Gini={gini:.3f})', lw=2)
    axes[1].plot([0,1], [0,1], 'k--', label='Perfect Equality')
    axes[1].set_title('b) Lorenz Curve for Wealth')
    axes[1].set_xlabel('Cumulative Share of Population'); axes[1].set_ylabel('Cumulative Share of Wealth')
    axes[1].legend()
    
    # 3. Plot wealth distribution histogram
    axes[2].bar(model.a_grid, model.dist_star[:, 0], width=model.a_grid[1]-model.a_grid[0], label='Low Income', alpha=0.7)
    axes[2].bar(model.a_grid, model.dist_star[:, 1], width=model.a_grid[1]-model.a_grid[0], bottom=model.dist_star[:, 0], label='High Income', alpha=0.7)
    axes[2].set_title('c) Stationary Wealth Distribution')
    axes[2].set_xlabel('Asset Level'); axes[2].set_ylabel('Mass of Households')
    axes[2].legend()
    
    for ax in axes: ax.grid(True)
    plt.tight_layout(rect=[0,0,1,0.96]); plt.show()

plot_aiyagari_results(aiyagari_model)

### Visualizing the Convergence to the Stationary Distribution
To make the concept of a stationary distribution more concrete, the following animation shows how the cross-sectional distribution of wealth evolves over time, starting from a point where all agents have zero wealth. The distribution initially spreads out as agents experience different income shocks and build up precautionary savings. Over time, the distribution converges to a stable, right-skewed shape, which is the model's stationary wealth distribution.

![Animation of the Aiyagari model's wealth distribution converging to its stationary state.](../images/10-Specialized-Models/aiyagari_wealth_distribution.gif)

<a id='importance'></a>
## 5. The Importance of the Aiyagari Model

The Aiyagari model was a landmark achievement in macroeconomics for several reasons:
1.  **Quantifying Precautionary Savings:** It was the first widely-used general equilibrium model to formalize and quantify the precautionary savings motive. It showed that adding uninsurable income risk could significantly increase the aggregate savings rate, helping to explain why aggregate wealth is so high.
2.  **Explaining Wealth Inequality:** The model generates a persistent, non-degenerate distribution of wealth from a model with ex-ante identical agents. The inequality arises endogenously from the history of income shocks each agent receives. This provides a powerful, disciplined laboratory for studying the sources of wealth inequality.
3.  **A Methodological Bridge:** It provided a tractable framework that bridged the gap between simple representative-agent models and more complex, less disciplined agent-based models. It retains the rigor of general equilibrium and rational expectations while incorporating the crucial feature of agent heterogeneity.
4.  **Foundation for Modern Macro:** The computational techniques developed to solve the Aiyagari model laid the foundation for the vast literature on heterogeneous agent macroeconomics that followed, including models with more realistic features like life-cycle dynamics, housing, and portfolio choice (HANK models).

<a id='summary'></a>
## 6. Summary and Key Takeaways

This notebook provided a complete walkthrough of the canonical Aiyagari (1994) heterogeneous agent model.
- Heterogeneous agent models are necessary to study questions of inequality and distribution.
- The Aiyagari model shows how **precautionary savings** in response to **uninsurable idiosyncratic risk** can generate a stationary distribution of wealth.
- Solving these models requires a nested-loop algorithm: an **outer loop** searches for market-clearing prices, and an **inner loop** solves the household's optimization problem for a given set of prices.
- The model's equilibrium is characterized by a savings policy function that shows households building a buffer stock of assets, and a skewed wealth distribution that, while a major improvement on representative-agent models, is still less concentrated than the wealth distribution seen in the data.

<a id='exercises'></a>
## 7. Exercises

1.  **The Role of Patience:** The household's discount factor, $\beta$, determines their patience. How does the equilibrium interest rate $r^*$ change if households become more patient (e.g., $\beta=0.98$)? Re-solve the model and explain the economic intuition for why the aggregate capital supply curve shifts.

2.  **Increased Risk:** The persistence of the low-income state is governed by the transition probability $\pi(y_L|y_L)$. How does the equilibrium change if this probability increases (e.g., to `y_trans = np.array([[0.7, 0.3], [0.3, 0.7]])`)? Does the equilibrium interest rate fall or rise? Why? What does this tell you about the power of the precautionary savings motive?

3.  **No Borrowing Constraint:** What happens to the model if we relax the borrowing constraint and allow `a_min = -2.0`? Try to solve the model. What problem do you encounter? (Hint: Think about households that receive the high income shock forever). What does this tell you about the importance of borrowing constraints in this class of models?

4.  **Incomplete vs. Complete Markets:** In the standard representative-agent neoclassical growth model, the equilibrium interest rate is determined solely by the condition $r = 1/\beta - 1$. How does the equilibrium interest rate in the Aiyagari model compare to this "complete markets" benchmark? Is it higher or lower? What does the difference represent?

5.  **Adding a Wealth Tax:** Augment the model to include a simple proportional wealth tax, $\tau$, that is collected and then redistributed to all households as a lump-sum transfer, $T$. The household's budget constraint becomes $c_t + a_{t+1} = (1+r(1-\tau))a_t + y_t + T$. Modify the code to solve for the new stationary equilibrium. How does a 1% wealth tax affect the Gini coefficient of the wealth distribution?